### Runge-Kutta-Fehlberg Method

This code implements the Runge-Kutta-Fehlberg Method to approximate ordinary differential equations of the form $y'(t) = f(t,y)$ at *adaptively-spaced* mesh points between endpoints $a$ and $b$. 

This code implements the method as described in Algorithm 5.3 of *Numerical Analysis* (10th Edition) by Burden and Faires. This notebook presently provides the solutions to approximations for 5.5.3(a) in the same book.

In [1]:
# Imports
import numpy as np
import pandas as pd
import math

# For more decimal places
pd.set_option("display.precision", 7)

Specify your arguments below.

In [2]:
# Function
f = lambda t, y: y/t - (y/t)**2
# Left endpoint
a = 1
# Right endpoint
b = 4
# Maximum step size
hmax = 0.5
# Minimum step size
hmin = 0.05
# Tolerance
TOL = pow(10, -6)
# Initial condition
alpha = 1

# Maximum number of steps (unknown a priori)
N_max = int((b-a)/hmin) + 1

# Initialize arrays
t = np.zeros(N_max)
w = np.zeros(N_max)
h_out = np.zeros(1)

# Initial conditions
t[0] = a
w[0] = alpha
h_out[0] = hmax
h = hmax
i = 0
FLAG = 1

In [3]:
# Implementing the RKF algorithm for approximations
while (FLAG == 1):
  K1 = h*f(t[i],w[i])
  K2 = h*f(t[i] + 1/4*h, w[i] + 1/4*K1)
  K3 = h*f(t[i] + 3/8*h, w[i] + 3/32*K1 + 9/32*K2)
  K4 = h*f(t[i] + 12/13*h, w[i] + 1932/2197*K1 - 7200/2197*K2 + 7296/2197*K3)
  K5 = h*f(t[i] + h, w[i] + 439/216*K1 - 8*K2 + 3680/513*K3 - 845/4104*K4)
  K6 = h*f(t[i] + 1/2*h, w[i] - 8/27*K1 + 2*K2 - 3544/2565*K3 + 1859/4104*K4 - 11/40*K5)

  h_out = np.append(h_out, h)
  R = (1/h)*abs(1/360*K1 - 128/4275*K3 - 2197/75240*K4 + 1/50*K5 + 2/55*K6)
  delta = 0.84*pow(TOL/R, 1/4)

  if R <= TOL:
    t[i+1] = t[i] + h
    w[i+1] = w[i] + 25/216*K1 + 1408/2565*K3 + 2197/4104*K4 - 1/5*K5
    i += 1

  delta = 0.84*pow(TOL/R, 1/4)

  if delta <= 0.1:
    h = 0.1*h
  elif delta >= 4:
    h = 4*h
  else:
    h = delta*h
  
  if h > hmax:
    h = hmax
  
  if t[i] >= b:
    FLAG = 0
  elif (t[i] + h) > b:
    h = b - t[i]
  elif h < hmin:
    FLAG = 0

# Posterior update to approximation arrays
t = t[:i+1]
w = w[:i+1]
h_out = h_out[2:]

In [4]:
# Actual solution
y = lambda t: t/(1+np.log(t))
yt = np.zeros(len(t))

# Looping to determine actual values
for idx, val in enumerate(t):
  yt[idx] = y(val)

In [5]:
# Output
df = pd.DataFrame({('tᵢ'): t, 'wᵢ': w, 'hᵢ': h_out, 'y(tᵢ)': yt})
df

,tᵢ,wᵢ,hᵢ,y(tᵢ)
0,1.0000000,1.0000000,0.1330513,1.0000000
1,1.1101946,1.0051237,0.1101946,1.0051237
2,1.2191314,1.0175212,0.1089368,1.0175211
3,1.3572694,1.0396749,0.1381381,1.0396749
4,1.5290112,1.0732757,0.1717417,1.0732756
5,1.7470584,1.1213948,0.2180472,1.1213947
6,2.0286416,1.1881702,0.2815832,1.1881700
7,2.3994350,1.2795396,0.3707934,1.2795395
8,2.8985147,1.4041843,0.4990798,1.4041842
9,3.3985147,1.5285639,0.5000000,1.5285638
